In [1]:
from pathlib import Path


from matplotlib import rcParams
from matplotlib.cm import rainbow

import numpy as np
import pandas as pd


from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn import  linear_model

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
# from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


def encoding_label(label):
  if(label=='correct'):
    return 1
  else:
    return 0

pd.set_option("display.max_columns", 100)

/kaggle/input/fare-classification/meta_data.csv
/kaggle/input/ml-dataset/ML data/sample_submission.csv
/kaggle/input/ml-dataset/ML data/test.csv
/kaggle/input/ml-dataset/ML data/train.csv


In [2]:
# !pip install -U -q PyDrive
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials

In [3]:
# # Authenticate and create the PyDrive client.
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

In [4]:
# train_data_link = "https://drive.google.com/open?id=18478S0JLw0ISKluNjkKcnRj2OcqBVCmA"
# test_data_link = "https://drive.google.com/open?id=1jdpYjY0eT6Qtfny8qOD6wbTj95T-h2mx"
# submissionFormat_link = "https://drive.google.com/open?id=19r1md8qztL2aYJJW5fu4Hxqhj3XLR5LQ"

# fluff, train_data_id = train_data_link .split('=')
# fluff, test_data_id = test_data_link.split('=')
# fluff, submissionFormat_id = submissionFormat_link.split('=')
# print (train_data_id,test_data_id,submissionFormat_id) # Verify that you have everything after '='

In [5]:
# thisDict = {"training_set":train_data_id,"test_set":test_data_id,"submission_format":submissionFormat_id}
# resultDic = {}
# for name,id in thisDict.items():
#   print (name, id)
#   downloaded = drive.CreateFile({'id':id}) 
#   downloaded.GetContentFile(name+'.csv')  
#   resultDic[name] = pd.read_csv(name+'.csv', index_col="tripid")

# train_data = resultDic["training_set"]
# train_data["label"]=train_data["label"].apply(encoding_label).values
# test_data = resultDic["test_set"]

train_data = pd.read_csv("/kaggle/input/ml-dataset/ML data/train.csv", index_col="tripid")
train_data["label"]=train_data["label"].apply(encoding_label).values
test_data = pd.read_csv("/kaggle/input/ml-dataset/ML data/test.csv", index_col="tripid")
submissionFormatData = pd.read_csv("/kaggle/input/ml-dataset/ML data/sample_submission.csv", index_col="tripid")
#training_data=train_data["label"]
# index_data=test_data["tripid"]
train_data.head()
# Trainng_feature=train_data[]
# trainng_feature= train_data.loc[:,[0:12]]
trainng_label= train_data[["label"]]
trainng_feature= train_data.loc[:,"additional_fare":"fare"]
# trainng_feature=trainng_feature.drop(['pickup_time','drop_time'],axis=1)
trainng_label.head()

,label
tripid,
189123628,1
189125358,1
189125719,1
189127273,1
189128020,1


Data Preprocessing 

In [6]:
numerical_cols = trainng_feature.columns[trainng_feature.dtypes != "object"].values

In [7]:
numeric_preprocessing_steps = Pipeline([
    ('standard_scaler', StandardScaler()),
    ('simple_imputer', SimpleImputer(strategy='mean'))
])

preprocessor = ColumnTransformer(
    transformers = [
        ("numeric", numeric_preprocessing_steps, numerical_cols)
    ],
    remainder = "drop"
)

In [8]:
estimators = MultiOutputClassifier(
    estimator=RandomForestClassifier(max_depth=10, random_state=0)
)

In [9]:
full_pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("estimators", estimators),
])

In [10]:
full_pipeline

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('numeric',
                                                  Pipeline(memory=None,
                                                           steps=[('standard_scaler',
                                                                   StandardScaler(copy=True,
                                                                                  with_mean=True,
                                                                                  with_std=True)),
                                                                  ('simple_imputer',
                                                                   SimpleImputer(add_indicator=False,
                                                                 

Creating the dataset

In [11]:
X_train, X_eval, y_train, y_eval = train_test_split(
    trainng_feature,
    trainng_label,
    test_size=0.3,
    shuffle=True,
    stratify= trainng_label,
    random_state=6
)
X_train.head()

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare
tripid,,,,,,,,,,,,
196858394,10.5,55.0,37.0,0.0000,410.0,11/30/2019 20:18,11/30/2019 20:19,6.85037,79.9221,6.84474,79.9314,60.50
191473740,10.5,1908.0,151.0,0.0000,215.0,11/12/2019 19:19,11/12/2019 19:51,6.99598,79.9028,6.92720,79.8443,449.33
195369258,35.0,2579.0,492.0,28.6836,0.0,11/27/2019 12:52,11/27/2019 13:35,6.86733,79.8758,6.78777,79.9002,592.52
204421493,10.5,744.0,83.0,0.0000,156.0,12/27/2019 10:50,12/27/2019 11:03,6.86711,79.8654,6.87280,79.8911,131.36
213456300,10.5,910.0,1.0,0.0000,228.0,1/31/2020 0:58,1/31/2020 1:13,6.84117,79.9654,6.87333,79.8876,357.19


Train the model

In [12]:
full_pipeline.fit(X_train, y_train)

# Predict on evaluation set
# This competition wants probabilities, not labels
preds = full_pipeline.predict(X_eval)
preds

array([[1],
       [1],
       [1],
       ...,
       [1],
       [1],
       [1]])

In [13]:
print("test_probas[0].shape", preds.shape)

test_probas[0].shape (5153, 1)


In [14]:
y_preds = pd.DataFrame(
    {
        "prediction": preds[:, 0]
    },
    index = y_eval.index
)
print("y_preds.shape:", y_preds.shape)
y_preds.head()

y_preds.shape: (5153, 1)


,prediction
tripid,
204424184,1
205888602,1
202764678,1
204205375,1
205876129,1


Test model

In [15]:
f1_score(y_eval, y_preds, average=None)

array([0.54912517, 0.96496915])

In [16]:
full_pipeline.fit(trainng_feature, trainng_label)

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('numeric',
                                                  Pipeline(memory=None,
                                                           steps=[('standard_scaler',
                                                                   StandardScaler(copy=True,
                                                                                  with_mean=True,
                                                                                  with_std=True)),
                                                                  ('simple_imputer',
                                                                   SimpleImputer(add_indicator=False,
                                                                 

In [17]:
test_probas = full_pipeline.predict(test_data)
test_probas

array([[1],
       [1],
       [1],
       ...,
       [1],
       [1],
       [1]])

In [18]:
# Make sure we have the rows in the same order
np.testing.assert_array_equal(test_data.index.values, 
                              submissionFormatData.index.values)


submissionFormatData["prediction"]=test_probas[:, 0]
submissionFormatData.head()

submissionFormatData.to_csv('my_submission.csv', index=True)

Show output

In [19]:
!head my_submission.csv

tripid,prediction
213284604,1
213286352,1
213293973,1
213294622,1
213298687,1
213299545,1
213302332,1
213302671,1
213305594,1
